<a href="https://colab.research.google.com/github/dudaspm/iPython/blob/master/R_Python_JavaScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# R -> Python -> JavaScript

In [9]:
## Enable R

In [10]:
%%capture
%load_ext rpy2.ipython

## Install R Package

In [11]:
%%capture 
%R install.packages("pwr")

## R Code

### Power Calculation For Two Proportions (Same Sample Sizes) 

[pwr.2p.test](https://www.rdocumentation.org/packages/pwr/versions/1.3-0/topics/pwr.2p.test)

In [12]:
%%R -o powerData
library(pwr)
no.of.samples=seq(10,100,10)
pvals = c(.01,.05)
effect_size_2_temp = {}
effect_size_2 = {}
pvalues = c()
samples = c()
functions = c()
power = c()
powerExp = seq(from=0.1, to = .99, by = 0.01)
for (pv in pvals) {
    for (x in no.of.samples){
        for (pe in powerExp) {
            effect_size_2_temp = (pwr.2p.test(n=x, sig.level = pv, power = pe))$h
            functions = c(functions,effect_size_2_temp)
            pvalues = c(pvalues,pv)
            samples = c(samples,x)
            power = c(power,pe)
        }
    }
}
powerData <- data.frame(samples,functions,pvalues,power)

## R -> Python

In [13]:
import pandas as pd
import json
outputToHTML = []
document = powerData.to_csv().split("\n");
del document[len(document)-1]
header = document[0].split(",")[1:5]
del document[0]
tuples = {}
for row in document:
    row = row.split(",")[1:5]
    sampleNum = row[0]
    linePoint = row[1]
    pval = row[2]
    power = row[3]
    if (pval,sampleNum) not in tuples:
        tuples[(pval,sampleNum)] = {}
        tuples[(pval,sampleNum)]["pval"] = pval
        tuples[(pval,sampleNum)]["sample"] = sampleNum
        tuples[(pval,sampleNum)]["points"] = []
    tuples[(pval,sampleNum)]["points"].append([power,linePoint])

In [14]:
j = "" # JSON object
j = j + "{"    
j = j + """\t"nodes": ["""
for t in tuples:
    j = j + str(json.dumps(tuples[t])) + ",\n"

j = j[:-2]
j = j + "\t]\n"
j = j + "}"


## Enable D3.js

In [15]:
from IPython.display import  HTML

def load_d3_in_cell_output():
  display(HTML("<script src='https://d3js.org/d3.v5.min.js'></script>"))
get_ipython().events.register('pre_run_cell', load_d3_in_cell_output)

## Python -> JavaScript

In [17]:
import IPython
js_code = \
'''

        let data = {};
        const height = 350;
        const width = 800;
        const svg = d3.select('div#output-area').append('svg')
            .attr("id","graph")
            .attr("width", width+"px")
            .attr("height", height+"px");

        const points = [];
        for (var i = 0; i < data.nodes.length; i++) for (var j = 0; j < data.nodes[i].points.length; j++) points.push(data.nodes[i].points[j]);
        svg.selectAll('circle')
            .data(data)
            .enter()
            .append('circle')
            .attr("cx", (d,i)=>40 * (i + 1))
            .attr("cy", (d,i)=>40 * (i + 1))
            .style("fill", "#1570a4")
            .transition().duration(2000)
            .attr("r", (d,i)=>d*2);


        window.x = d3.scaleLinear().rangeRound([50, width-50]).domain(d3.extent(points,d=>d[1]));
        window.y = d3.scaleLinear().rangeRound([height-50, 50]).domain(d3.extent(points,d=>d[0]));
        const color = d3.scaleLinear().range([0,1]).domain([0,89]);
        const palette = d3.interpolateBlues;
        const scale = d3.scaleOrdinal(d3.schemeCategory10);

        svg.append("g").attr("transform", "translate(0," + (height-50) + ")").call(d3.axisBottom(x))
            .append("text")             
            .attr("transform", "translate(" + (width/2) + " ," + (40) + ")")
            .style("text-anchor", "middle")
            .style("fill", "#000")
            .style("font-size", "16")
            .text("Effect Size");
        svg.append("g").attr("transform", "translate(50,0)").call(d3.axisLeft(y))
            .append("text")
            .attr("transform", "rotate(-90)")
            .attr("y", -50)
            .attr("x",0 - (height / 2))
            .attr("dy", "1.25em")
            .style("text-anchor", "middle")
            .style("fill", "#000")
            .style("font-size", "16")
            .text("Power"); 

        const firstplot = data.nodes.filter(d=> ((d.pval == .01) && (d.sample == 10)) )
        svg.selectAll("g.links").data(firstplot).enter().append("g")
            .attr("class","links")
            .selectAll("line.area").data(d=> d.points).enter().append("line")
            .attr("class", "area")
            .attr("x1", d=> x(d[1]) )
            .attr("y1", d=> y(d[0]) )
            .attr("x2", d=> x(d[1]) )
            .attr("y2", d=> y(d[0]) )
            .style("stroke", "steelblue")
            .style("stroke-width", "1px") 

        const node = svg.selectAll("g.node").data(firstplot).enter().append("g")
            .attr("class","node")

            .style("stroke-width", 1.5)
            .selectAll("circle").data(d=> d.points)
                
            .enter().append("circle")
            .attr("class","degree")
            .attr("r", d => 5)
            .attr("cx", d => x(d[1]))
            .attr("cy", d => y(d[0]))
            .style("stroke", "black")

            .style("fill", "black")

          var sampleSize = 10;
          svg.append("text")
            .attr("id","pop")
            .attr("x",width-350)
            .attr("y", height-100)
            .style("font-size","30px")
            .text("Population Size: "+sampleSize)
          
        
          
          setInterval(() => {{
            sampleSize = (sampleSize==100) ? 10 : sampleSize+10;
            svg.select("text#pop").text("Population Size: "+sampleSize)
            const updatePlot = data.nodes.filter(d=> ((d.pval == .01)&&(d.sample == parseInt(sampleSize))) ) 
            svg.selectAll("g.node").data(updatePlot)
                .selectAll("circle").data(d=> d.points)
                .transition("2")
                .duration(1000)
                .delay((d,i) => i * 25 )
                .attr("cx", d => x(d[1]) )
                .attr("cy", d => y(d[0]) )
              svg.selectAll("g.links").data(updatePlot)
                .selectAll("line.area").data(d=> d.points )
                .transition("area").duration(1000)
                .delay((d,i)=> i * 25 )
                .attr("x1", function() {{ return d3.select(this).attr("x1") }})
                .attr("y1", function() {{ return d3.select(this).attr("y1") }})
                .attr("x2", d => x(d[1]) )
                .attr("y2", d => y(d[0]) )
              

          }}, 5000);



'''.format(j)


display(IPython.display.Javascript(js_code))

<IPython.core.display.Javascript object>